<a href="https://colab.research.google.com/github/RepedeMonica/Neural-Networks-Template-2024/blob/main/NN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

loading the data

In [1]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import Dataset

image_to_tensor = transforms.Compose([transforms.ToTensor()])
train_dataset = MNIST('data', train=True, transform=image_to_tensor, download=True)
test_dataset = MNIST('data', train=False, transform=image_to_tensor, download=True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.2MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 725kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.77MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.6MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



DATASETS

In [2]:
#DATASETS
class MNISTDataset(Dataset):
    def __init__(self, dataset):
        self.data = [dataset[i][0] for i in range(len(dataset))]
        self.labels = [dataset[i][1] for i in range(len(dataset))]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        data = self.data[i].view(-1)
        label = self.labels[i]
        return data, label
dataset = MNISTDataset(train_dataset)
val_dataset = MNISTDataset(test_dataset)

DataLoader

In [3]:
#DATALOADERS
from torch.utils.data import DataLoader
dataset_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=100, shuffle=False)


MY MODEL

In [4]:
#MODEL
from torch import nn
from torch import Tensor
class MyNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MyNN,self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size1)
        self.layer_2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer_3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()

    def forward(self, x: Tensor):
        # x is a Tensor of size [batch size, input_size]
        x = self.layer_1(x) # x is a Tensor of size [batch size, hidden_size]
        x = self.relu(x)  # We can use any activation function

        x = self.layer_2(x)
        x = self.relu(x)

        x = self.layer_3(x)
        return x
model = MyNN(input_size=784, hidden_size1=128, hidden_size2=64, output_size=10)
print(model)

MyNN(
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)


DEVICES

In [5]:
#DEVICES
import torch
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')  # On multi-gpu workstation we can select cuda:0, cuda:1, ...
    if torch.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')
device = get_device()
print(device)
model = model.to(device)


cuda


OPTIMIZERS

In [6]:
#OPTIMIZATOR
# Optimizers apply the gradients calculated by the Autograd engine to the weights, using their own optimization technique
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True,weight_decay=0.001)  # SGD with Nesterov momentum and weight decay
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)  # Adam with Weight Decay
# Schedulers change the learning rate enabling faster training
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10,gamma=0.5)  # Learning rate scheduler, halves the learning rate each 10 steps


LOSS *FUNCTION*

In [7]:
#LOSS
criterion = nn.CrossEntropyLoss()  # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html


TRAIN

In [8]:
#TRAIN
import csv
def train(model, train_dataloader, criterion, optimizer, device):
    model.train()  # We need to activate the dropout & batch norm layers

    mean_loss = 0.0

    for data, labels in train_dataloader:
        data = data.to(device)  # We move the data to device. Bonus: we can do this in an async manner using non_blocking and pin_memory
        labels = labels.to(device)

        outputs = model(data)  # the forward pass
        loss = criterion(outputs, labels)  # we calculate the loss

        optimizer.zero_grad() # reset the gradients
        loss.backward()  # we backpropagate the loss

        optimizer.step()  # we update the weights

        mean_loss += loss.item()

    mean_loss /= len(train_dataloader)
    return mean_loss


validation

In [9]:
@torch.inference_mode()  # it is better to decorate the method with torch.inference_mode or torch.no_grad
def val(model, val_dataloader, criterion, device):
    num_correct = 0
    num_samples = 0
    mean_loss = 0.0
    model.eval()  # We need to deactivate the dropout & batch norm layers

    for data, labels in val_dataloader:
        data = data.to(device)
        labels = labels.to(device)

        output = model(data)
        loss = criterion(output, labels) # we calculate the loss
        _, predicted = output.max(1)
        num_correct += (predicted == labels).sum()
        num_samples += predicted.size(0)  # size of the mini batch

        mean_loss += loss.item()

    mean_loss /= len(val_dataloader)
    return mean_loss, num_correct / num_samples


GENERATE PREDICTIONS TO CSV

In [10]:
@torch.inference_mode()
def generate_predictions_csv(model, val_dataloader, device, output_csv="submisions.csv"):
    model.eval()
    predictions = []
    correct_predictions = 0
    total_predictions = 0

    for i, (data, labels) in enumerate(val_dataloader):
        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        _, predicted = outputs.max(1)

        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)  #

        for idx, pred in enumerate(predicted):
            predictions.append((i * val_dataloader.batch_size + idx, pred.item()))

    accuracy = correct_predictions / total_predictions * 100
    print(f"Final test accuracy: {accuracy:.2f}%")

    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "target"])
        writer.writerows(predictions)

    print(f"Predictions were saved in {output_csv}")



traininf loop

In [11]:
#TRAIN LOOP
from tqdm import tqdm
def main(model, train_dataloader, val_dataloader, criterion, optimizer, device, epochs):
    with tqdm(tuple(range(epochs))) as tbar:
        for epoch in tbar:
            train_loss = train(model, train_dataloader, criterion, optimizer, device)
            val_loss, accuracy = val(model, val_dataloader, criterion, device)
            scheduler.step()
            tbar.set_description(f"Train loss: {train_loss:.5f} | Val loss: {val_loss:.5f} | Val accuracy: {accuracy:.5f}")
        generate_predictions_csv(model, val_dataloader, device)


main(model, dataset_loader, val_dataloader, criterion, optimizer, device, 150)


Train loss: 0.03086 | Val loss: 0.06210 | Val accuracy: 0.98230: 100%|██████████| 150/150 [04:45<00:00,  1.90s/it]


Final test accuracy: 98.23%
Predictions were saved in submisions.csv
